# Creating a map with F1 information

## Circuits, Constructors and Drivers around the Globe

Installing the required modules

In [1]:
#pip install wikipedia
#pip install folium

In [2]:
import folium #module to create maps
import pandas as pd #data science module
import pyodbc #to connect to azure sql
from decimal import * #to handle wikipedia coordinates

In [3]:
import wikipedia as wiki

Connecting python to the Azure db.

In [4]:
server = 'f1server.database.windows.net'
database = 'WackyRacesF1'
username = 'PDS'
password = '{Formula1}'   
driver= '{ODBC Driver 17 for SQL Server}'

Running SQL queries using Python to import the required information into Pandas dataframes.

In [5]:
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
constructors_query = "SELECT DISTINCT c.constructorId, constructorRef, c.name, nationality, c.url FROM constructors c LEFT JOIN constructorResults cr ON c.constructorId = cr.constructorId LEFT JOIN races r ON r.raceId = cr.raceId WHERE year >= 2015 and year <= 2022;"
constructors = pd.read_sql(constructors_query, cnxn)
circuits_query = "SELECT DISTINCT c.circuitId, c.name, c.location, c.country, lat, lng, alt, c.url FROM circuits c LEFT JOIN races r ON r.circuitId = c.circuitId WHERE year >= 2015 and year <= 2022;"
circuits = pd.read_sql(circuits_query, cnxn)

Importing csv with information on drivers into an additional Pandas dataframe.

In [6]:
drivers = pd.read_excel('f1_drivers_2015_2021.xlsx')

### Creating functions to extract information from wikipedia

In [7]:
def get_wiki_summary(df):
    summary = []

    for name in df:
        try:
            detail = wiki.WikipediaPage(title=name).summary
            summary.append(detail)
        except:
            summary.append('error')
            print(name)
        
    return summary

In [46]:
def get_coords(df):
    lat = []
    long = []

    for name in df:
        try:
            coord = wiki.WikipediaPage(title=name).coordinates
            lat.append(coord[0])
            long.append(coord[1])
        except:
            if name == "Nürburgring": #to account for the Nürburgring
                lat.append(Decimal('50.57469354557302'))
                long.append(Decimal('7.045744986271267'))
            elif name == "Miami International Autodrome": #to account for the Miami International Autodrome
                lat.append(Decimal('25.95717260961372'))
                long.append(Decimal('-80.231329321313'))
            elif name == "Netherlands": #to account for the Netherlands
                lat.append(Decimal('52.36463903490765')) 
                long.append(Decimal('4.908115191408701'))
            elif name == "Russia": #to account for Russia
                lat.append(Decimal('55.802479274214974'))
                long.append(Decimal('37.592885102145694'))
            else:
                lat.append(Decimal('3'))
                long.append(Decimal('4'))
                print(name)
                
    return lat, long

In [62]:
circuits.head(1)

,circuitId,name,location,country,alt,url,summary,lat,long
0,1,Albert Park Grand Prix Circuit,Melbourne,Australia,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,The Albert Park Circuit is a motorsport street...,-37.849722219999996752903825836256146430969238...,144.96833333000000720858224667608737945556640625


In [63]:
drivers.head(1)

,DRIVER_ID,DRIVER,DRIVER_REF,DRIVER_CODE,DOB,NATIONALITY,COUNTRY,SUMMARY,lat,long
0,1,Lewis Hamilton,hamilton,HAM,1985-01-07,British,United Kingdom,Sir Lewis Carl Davidson Hamilton (born 7 Janu...,55,-3


In [91]:
constructors.head(1)

,constructorId,constructorRef,name,nationality,url,country,lat,long
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,United Kingdom,55,-3


In [109]:
def generate_map(circuits, drivers, constructor):
    
    my_map = folium.Map(location=[38.73241897525329, -9.160338893258317], #the coordinates to Nova IMS serve as the initial focal point for the map
                    zoom_start=2,
                    tiles='CartoDB dark_matter',
                    attr='João Rodrigues, Nuno Lopes, Ricardo Almeida, Soraia Alves, Teresa Barrosa'
                   )
    
    #inputting circuit information into the map
    for index, row in circuits.iterrows():
        
        html=f"""
        <h1><img src='data:image/svg+xml;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz4KPCEtLSBHZW5lcmF0b3I6IEFkb2JlIElsbHVzdHJhdG9yIDI1LjIuMywgU1ZHIEV4cG9ydCBQbHVnLUluIC4gU1ZHIFZlcnNpb246IDYuMDAgQnVpbGQgMCkgIC0tPgo8c3ZnIHZlcnNpb249IjEuMSIgaWQ9IkxheWVyXzEiIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyIgeG1sbnM6eGxpbms9Imh0dHA6Ly93d3cudzMub3JnLzE5OTkveGxpbmsiIHg9IjBweCIgeT0iMHB4IgoJIHZpZXdCb3g9IjAgMCAxNTQ2LjIgOTcwLjIiIHN0eWxlPSJlbmFibGUtYmFja2dyb3VuZDpuZXcgMCAwIDE1NDYuMiA5NzAuMjsiIHhtbDpzcGFjZT0icHJlc2VydmUiPgo8c3R5bGUgdHlwZT0idGV4dC9jc3MiPgoJLnN0MHtmaWxsOm5vbmU7c3Ryb2tlOiMzNzQxNTE7c3Ryb2tlLXdpZHRoOjcuODU0NTtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQoJLnN0MXtmaWxsOm5vbmU7c3Ryb2tlOiNFRjQ0NDQ7c3Ryb2tlLXdpZHRoOjEuOTYzNjtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQoJLnN0MntmaWxsOm5vbmU7c3Ryb2tlOiMzQjgyRjY7c3Ryb2tlLXdpZHRoOjEuOTYzNjtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQoJLnN0M3tmaWxsOm5vbmU7c3Ryb2tlOiNGQkJGMjQ7c3Ryb2tlLXdpZHRoOjEuOTYzNjtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQo8L3N0eWxlPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDAiIGQ9Ik01NTMuOCwyMjkuOGMzMy4xLDY2LjUsNjIuNiwxMzUuMiw5NC4zLDIwMi4zCgljMTAuNCwyMi4xLDIwLjMsNDUuNSwzMi40LDY2LjhjOC40LDE4LjYsMTguNCwzOSwzMy40LDUzYzEyLjcsMTUuMywzNi4yLDE5LjIsNTUsMTUuN2MxNy0zLjUsMzIuNC0xMi44LDQ3LjEtMjEuNgoJYzIyLjYtMTQuOCw0Ni40LTI4LjgsNjguNy00NC4yYzEwOS42LTc0LjMsMjguMy0xMzMsODMuNS0yMThjMjMuOS0zOC42LDY4LjktNjMsMTEzLjktNjUuOGMyMC44LTMuMSw0My0wLjcsNjMuOC0yLjkKCWMyNS4yLTEuOSw1MC40LTcuOCw3NC42LTE0LjdjNDAuNy0xMi45LDgwLjEtMzEuOCwxMTguOC00OS4xYzIxLjYtMTIuOCw1MC41LTE4LjIsNjEuOC00Mi4yYzExLjMtMjYuOC0wLjEtNTEuMSwzNy4zLTYwLjkKCWMxMy45LTMsMjcuNCwyLjYsNDAuMyw1LjljMTMuMSwzLjMsMjYuMyw0LjQsMzkuMywzLjljMTguMy00LjMsMjctMjUuMiwxMC44LTM5LjNjLTExLjUtOC42LTI2LjktOS45LTQwLjMtOS44CgljLTIzMiw3LjQtNDY1LjIsNC4zLTY5Ny4xLDcuOWMtMTQyLjYsMC43LTI4NSwxLjMtNDI3LjEsMWMtMTgsMC42LTMxLDE0LjctMjguNSwzMi40YzguMywxNi4xLDE2LjEsMzUuMiwzLjksNTEuMQoJYy01LjQsMTAuOC0xOC4zLDE2LjItMjcuNSwyMi42Yy0xNy42LDExLjctMzcuMywyMC42LTUzLDM0LjRjLTI1LjcsMTguNS00My43LDQ1LjItNjAuOSw3MC43Yy0yOC4yLDQyLjMtNDguOSw4OC43LTY2LjgsMTM1LjUKCUM3Ni44LDUwMy41LDMwLjQsNjQ3LjYsMTcuOCw3OTYuNGMtMy40LDM0LTguOSw2Ny4xLTguOCwxMDAuMWM1LjQsMzMuOSwyNS42LDMzLjksNTQsMzUuM2MyOC41LDguNywzMiwyOC45LDYzLjgsMjkuNAoJYzE5LjUtMC40LDM4LjctNy41LDU2LTEzLjhjMTkuMS02LjUsMzguNy0xOC4xLDQzLjItMzguM2M1LjUtMzIuNC0yMC4yLTY0LjctMzYuMy05MC4zYy0xNy4xLTI4LjUtMzQuMy01Ny4xLTUwLjEtODUuNAoJYy01MS4xLTg0LjQtMjYtMjA2LjIsNTktMjU3LjJjMjAuOC0xNi4xLDU1LjUtMzcuMiw3NC42LTkuOGM3LjYsMTIuNSw5LjMsMjcuMiwxMy43LDQwLjNjNS41LDIzLjksMjguMSw0MS44LDUyLjEsMzkuMgoJYzE5LjctMi4xLDM5LjMtOS42LDU2LTE4LjdjMzYuMy0xOC42LDMxLjEtNjUuMSwxNC43LTk1LjNjLTE4LjMtNDAuMi00Ni43LTc0LjktNjYuOC0xMTIuOWMtMTcuOC0yOC4yLTM3LjItNjUtMTkuNi05Ni4yCgljNS4yLTE1LjcsMTcuOC0yNy41LDI4LjUtMzguM2M5LjMtMTEsMTktMjIsMjguNS0zMS40YzI0LjUtMjguMiw1NS45LTcxLDk3LjItNTMuOWMyNy41LDEzLjksMzkuNCw0NS41LDUwLjEsNzEuNwoJQzUzNS4yLDE5MS41LDU0NS4xLDIxMC4yLDU1My44LDIyOS44eiIvPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDEiIGQ9Ik01NTMuOCwyMjkuOGMzNi44LDc0LjQsNjkuOSwxNTAuOCwxMDUuMSwyMjUuOAoJYzMzLjQsNjYuMiw1NywxNTAuMywxNDcuMyw5Ni4yYzUxLTM1LjEsMTIxLTYxLDEzOC40LTEyNi43YzYtMjcuNSwwLjMtNTUuOCwyLTgzLjVjNS45LTQ0LjcsMzQuMi04NC41LDczLjYtMTA2CgljNDMuMS0yNS40LDk1LTE1LjgsMTQyLjQtMjIuNmM1Ny4yLTcuNCwxMTEuMS0zMS42LDE2My01NiIvPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDIiIGQ9Ik0xMzI1LjUsMTU3LjJjMTMuOC01LjcsMjgtMTIuOSw0MS4yLTE5LjZjMTMuOC01LjcsMjguMS0xNC4zLDM0LjQtMjguNQoJYzctMTIuOSw0LTI5LjEsMTAuOC00Mi4yYzcuMS0xNCwyNi4xLTIxLjksNDEuMi0xOC43YzkuOS0wLjQsMTkuNyw1LjQsMjkuNSw2LjljMTMsMS45LDI3LjYsNS44LDQwLjMsMQoJYzI1LjgtMTEuNiwxNC41LTM5LjUtMTAuOC00NS4yYy0xNi40LTMuMy0zMy41LTEuNi01MC4xLTFjLTIxLjIsMC4yLTQyLjcsMS02My44LDJjLTMzOS45LDIuOC02ODAsNi4zLTEwMjAuMSw1LjkKCWMtMjYuNS0yLjktNTEsMTAuOC0zOS4zLDQwLjNjMjIuMiwzNC4yLTcuOSw1Ny0zNi4zLDcxLjdjLTM0LDE5LjQtNjYuOSw0My04OC40LDc2LjZjLTQyLjUsNTctNzAuMywxMjMtOTQuMywxODkuNQoJQzcwLjYsNTIxLjMsMzEuOCw2NTIsMTguNyw3ODYuNSIvPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDMiIGQ9Ik0xOC43LDc4Ni41Yy0yLjksMjMuNi00LjgsNDguMi03LjksNzEuN2MtMS41LDIyLTcuNiw0OS41LDExLjgsNjUuOAoJYzEwLDcuMSwyMi42LDcuMSwzNC40LDYuOWM3LjksMC44LDE1LjQsNSwyMi42LDcuOWMyNy42LDI1LjksNTAuNCwyNy40LDg1LjQsMTQuN2M0MC43LTEwLjUsNzYuOC0zMi44LDU2LTc5LjUKCWMtMTAuMy0yNi45LTI5LjItNDkuNy00My4yLTc0LjZjLTIwLjMtMzYuNC00Ni4zLTcxLjQtNTcuOS0xMTEuOWMtMTIuNS00NC40LTYuNS05My4xLDEwLjgtMTM1LjVjMjAuMi00MS45LDU3LjMtNzMuNSw5OC4yLTk0LjMKCWMyMi4yLTExLjgsNDcuMSwwLjQsNTEuMSwyNS41YzcuOCwzMCwxNiw2MS4yLDUzLDYyLjhjMTQuNiwwLjksMjguOS00LjgsNDIuMi05LjhjMjQtNi41LDQ2LjUtMjUuOSw0Ny4xLTUyCgljMS4yLTMzLjUtMTcuNS02My42LTM0LjQtOTEuM2MtMTcuNS0yOC4zLTM2LjEtNTYuMi01Mi04NS40Yy0xOS43LTMxLTI4LjUtNjYuNC01LjktOTguMmMxOC4yLTIzLDM5LjItNDQuMSw1OC45LTY1LjgKCWMxMi44LTE2LDI3LjYtMzEuNiw0NS4yLTQyLjJjMTkuNi05LjIsNDYuNC03LjcsNTkuOSwxMC44YzE2LjYsMTUuOCwyNSwzOC4xLDMzLjQsNTguOWM3LjYsMTkuOSwxNy44LDM5LjQsMjYuNSw1OC45Ii8+Cjwvc3ZnPgo='/></p>
        </h1>
        <h1><p> {row['name']}</p></h1>
        <p><p>Circuit information:</p>
        <ul>
            <li>Country name: {row['country']}</li>
            <li>City name: {row['location']}</li>
        <details>
            <summary>More details</summary>
            {row['summary']}
        </details>     
        </ul>
        </p>
        <p>To know more: <a href={row['url']}>link</a></p>
        """
        
        iframe = folium.IFrame(html=html, width=300, height=300)
        
        popup = folium.Popup(iframe, max_width=2650)
        
        folium.Marker(location=[row['lat'],row['long']],popup=popup,tooltip=row['name'],radius=3, icon=folium.Icon(color='red')).add_to(my_map)
        
    #inputting drivers information into the map
    for index, row in drivers.iterrows():
        
        html=f"""
        <h1><img src='data:image/svg+xml;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz4KPCEtLSBHZW5lcmF0b3I6IEFkb2JlIElsbHVzdHJhdG9yIDI1LjIuMywgU1ZHIEV4cG9ydCBQbHVnLUluIC4gU1ZHIFZlcnNpb246IDYuMDAgQnVpbGQgMCkgIC0tPgo8c3ZnIHZlcnNpb249IjEuMSIgaWQ9IkxheWVyXzEiIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyIgeG1sbnM6eGxpbms9Imh0dHA6Ly93d3cudzMub3JnLzE5OTkveGxpbmsiIHg9IjBweCIgeT0iMHB4IgoJIHZpZXdCb3g9IjAgMCAxNTQ2LjIgOTcwLjIiIHN0eWxlPSJlbmFibGUtYmFja2dyb3VuZDpuZXcgMCAwIDE1NDYuMiA5NzAuMjsiIHhtbDpzcGFjZT0icHJlc2VydmUiPgo8c3R5bGUgdHlwZT0idGV4dC9jc3MiPgoJLnN0MHtmaWxsOm5vbmU7c3Ryb2tlOiMzNzQxNTE7c3Ryb2tlLXdpZHRoOjcuODU0NTtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQoJLnN0MXtmaWxsOm5vbmU7c3Ryb2tlOiNFRjQ0NDQ7c3Ryb2tlLXdpZHRoOjEuOTYzNjtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQoJLnN0MntmaWxsOm5vbmU7c3Ryb2tlOiMzQjgyRjY7c3Ryb2tlLXdpZHRoOjEuOTYzNjtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQoJLnN0M3tmaWxsOm5vbmU7c3Ryb2tlOiNGQkJGMjQ7c3Ryb2tlLXdpZHRoOjEuOTYzNjtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQo8L3N0eWxlPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDAiIGQ9Ik01NTMuOCwyMjkuOGMzMy4xLDY2LjUsNjIuNiwxMzUuMiw5NC4zLDIwMi4zCgljMTAuNCwyMi4xLDIwLjMsNDUuNSwzMi40LDY2LjhjOC40LDE4LjYsMTguNCwzOSwzMy40LDUzYzEyLjcsMTUuMywzNi4yLDE5LjIsNTUsMTUuN2MxNy0zLjUsMzIuNC0xMi44LDQ3LjEtMjEuNgoJYzIyLjYtMTQuOCw0Ni40LTI4LjgsNjguNy00NC4yYzEwOS42LTc0LjMsMjguMy0xMzMsODMuNS0yMThjMjMuOS0zOC42LDY4LjktNjMsMTEzLjktNjUuOGMyMC44LTMuMSw0My0wLjcsNjMuOC0yLjkKCWMyNS4yLTEuOSw1MC40LTcuOCw3NC42LTE0LjdjNDAuNy0xMi45LDgwLjEtMzEuOCwxMTguOC00OS4xYzIxLjYtMTIuOCw1MC41LTE4LjIsNjEuOC00Mi4yYzExLjMtMjYuOC0wLjEtNTEuMSwzNy4zLTYwLjkKCWMxMy45LTMsMjcuNCwyLjYsNDAuMyw1LjljMTMuMSwzLjMsMjYuMyw0LjQsMzkuMywzLjljMTguMy00LjMsMjctMjUuMiwxMC44LTM5LjNjLTExLjUtOC42LTI2LjktOS45LTQwLjMtOS44CgljLTIzMiw3LjQtNDY1LjIsNC4zLTY5Ny4xLDcuOWMtMTQyLjYsMC43LTI4NSwxLjMtNDI3LjEsMWMtMTgsMC42LTMxLDE0LjctMjguNSwzMi40YzguMywxNi4xLDE2LjEsMzUuMiwzLjksNTEuMQoJYy01LjQsMTAuOC0xOC4zLDE2LjItMjcuNSwyMi42Yy0xNy42LDExLjctMzcuMywyMC42LTUzLDM0LjRjLTI1LjcsMTguNS00My43LDQ1LjItNjAuOSw3MC43Yy0yOC4yLDQyLjMtNDguOSw4OC43LTY2LjgsMTM1LjUKCUM3Ni44LDUwMy41LDMwLjQsNjQ3LjYsMTcuOCw3OTYuNGMtMy40LDM0LTguOSw2Ny4xLTguOCwxMDAuMWM1LjQsMzMuOSwyNS42LDMzLjksNTQsMzUuM2MyOC41LDguNywzMiwyOC45LDYzLjgsMjkuNAoJYzE5LjUtMC40LDM4LjctNy41LDU2LTEzLjhjMTkuMS02LjUsMzguNy0xOC4xLDQzLjItMzguM2M1LjUtMzIuNC0yMC4yLTY0LjctMzYuMy05MC4zYy0xNy4xLTI4LjUtMzQuMy01Ny4xLTUwLjEtODUuNAoJYy01MS4xLTg0LjQtMjYtMjA2LjIsNTktMjU3LjJjMjAuOC0xNi4xLDU1LjUtMzcuMiw3NC42LTkuOGM3LjYsMTIuNSw5LjMsMjcuMiwxMy43LDQwLjNjNS41LDIzLjksMjguMSw0MS44LDUyLjEsMzkuMgoJYzE5LjctMi4xLDM5LjMtOS42LDU2LTE4LjdjMzYuMy0xOC42LDMxLjEtNjUuMSwxNC43LTk1LjNjLTE4LjMtNDAuMi00Ni43LTc0LjktNjYuOC0xMTIuOWMtMTcuOC0yOC4yLTM3LjItNjUtMTkuNi05Ni4yCgljNS4yLTE1LjcsMTcuOC0yNy41LDI4LjUtMzguM2M5LjMtMTEsMTktMjIsMjguNS0zMS40YzI0LjUtMjguMiw1NS45LTcxLDk3LjItNTMuOWMyNy41LDEzLjksMzkuNCw0NS41LDUwLjEsNzEuNwoJQzUzNS4yLDE5MS41LDU0NS4xLDIxMC4yLDU1My44LDIyOS44eiIvPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDEiIGQ9Ik01NTMuOCwyMjkuOGMzNi44LDc0LjQsNjkuOSwxNTAuOCwxMDUuMSwyMjUuOAoJYzMzLjQsNjYuMiw1NywxNTAuMywxNDcuMyw5Ni4yYzUxLTM1LjEsMTIxLTYxLDEzOC40LTEyNi43YzYtMjcuNSwwLjMtNTUuOCwyLTgzLjVjNS45LTQ0LjcsMzQuMi04NC41LDczLjYtMTA2CgljNDMuMS0yNS40LDk1LTE1LjgsMTQyLjQtMjIuNmM1Ny4yLTcuNCwxMTEuMS0zMS42LDE2My01NiIvPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDIiIGQ9Ik0xMzI1LjUsMTU3LjJjMTMuOC01LjcsMjgtMTIuOSw0MS4yLTE5LjZjMTMuOC01LjcsMjguMS0xNC4zLDM0LjQtMjguNQoJYzctMTIuOSw0LTI5LjEsMTAuOC00Mi4yYzcuMS0xNCwyNi4xLTIxLjksNDEuMi0xOC43YzkuOS0wLjQsMTkuNyw1LjQsMjkuNSw2LjljMTMsMS45LDI3LjYsNS44LDQwLjMsMQoJYzI1LjgtMTEuNiwxNC41LTM5LjUtMTAuOC00NS4yYy0xNi40LTMuMy0zMy41LTEuNi01MC4xLTFjLTIxLjIsMC4yLTQyLjcsMS02My44LDJjLTMzOS45LDIuOC02ODAsNi4zLTEwMjAuMSw1LjkKCWMtMjYuNS0yLjktNTEsMTAuOC0zOS4zLDQwLjNjMjIuMiwzNC4yLTcuOSw1Ny0zNi4zLDcxLjdjLTM0LDE5LjQtNjYuOSw0My04OC40LDc2LjZjLTQyLjUsNTctNzAuMywxMjMtOTQuMywxODkuNQoJQzcwLjYsNTIxLjMsMzEuOCw2NTIsMTguNyw3ODYuNSIvPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDMiIGQ9Ik0xOC43LDc4Ni41Yy0yLjksMjMuNi00LjgsNDguMi03LjksNzEuN2MtMS41LDIyLTcuNiw0OS41LDExLjgsNjUuOAoJYzEwLDcuMSwyMi42LDcuMSwzNC40LDYuOWM3LjksMC44LDE1LjQsNSwyMi42LDcuOWMyNy42LDI1LjksNTAuNCwyNy40LDg1LjQsMTQuN2M0MC43LTEwLjUsNzYuOC0zMi44LDU2LTc5LjUKCWMtMTAuMy0yNi45LTI5LjItNDkuNy00My4yLTc0LjZjLTIwLjMtMzYuNC00Ni4zLTcxLjQtNTcuOS0xMTEuOWMtMTIuNS00NC40LTYuNS05My4xLDEwLjgtMTM1LjVjMjAuMi00MS45LDU3LjMtNzMuNSw5OC4yLTk0LjMKCWMyMi4yLTExLjgsNDcuMSwwLjQsNTEuMSwyNS41YzcuOCwzMCwxNiw2MS4yLDUzLDYyLjhjMTQuNiwwLjksMjguOS00LjgsNDIuMi05LjhjMjQtNi41LDQ2LjUtMjUuOSw0Ny4xLTUyCgljMS4yLTMzLjUtMTcuNS02My42LTM0LjQtOTEuM2MtMTcuNS0yOC4zLTM2LjEtNTYuMi01Mi04NS40Yy0xOS43LTMxLTI4LjUtNjYuNC01LjktOTguMmMxOC4yLTIzLDM5LjItNDQuMSw1OC45LTY1LjgKCWMxMi44LTE2LDI3LjYtMzEuNiw0NS4yLTQyLjJjMTkuNi05LjIsNDYuNC03LjcsNTkuOSwxMC44YzE2LjYsMTUuOCwyNSwzOC4xLDMzLjQsNTguOWM3LjYsMTkuOSwxNy44LDM5LjQsMjYuNSw1OC45Ii8+Cjwvc3ZnPgo='/></p>
        </h1>
        <h1><p> {row['DRIVER']}</p></h1>
        <p><p>Driver information:</p>
        <ul>
            <li>Country name: {row['COUNTRY']}</li>
        <details>
            <summary>More details</summary>
            {row['SUMMARY']}
        </details>     
        </ul>
        </p>
        """
        
        iframe = folium.IFrame(html=html, width=300, height=300)
        
        popup = folium.Popup(iframe, max_width=2650)
        
        folium.Marker(location=[row['lat'],row['long']],popup=popup,tooltip=row['SUMMARY'],radius=3, color='red').add_to(my_map)
        
        #inputting constructor information into the map
    for index, row in constructor.iterrows():
        
        html=f"""
        <h1><img src='data:image/svg+xml;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz4KPCEtLSBHZW5lcmF0b3I6IEFkb2JlIElsbHVzdHJhdG9yIDI1LjIuMywgU1ZHIEV4cG9ydCBQbHVnLUluIC4gU1ZHIFZlcnNpb246IDYuMDAgQnVpbGQgMCkgIC0tPgo8c3ZnIHZlcnNpb249IjEuMSIgaWQ9IkxheWVyXzEiIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyIgeG1sbnM6eGxpbms9Imh0dHA6Ly93d3cudzMub3JnLzE5OTkveGxpbmsiIHg9IjBweCIgeT0iMHB4IgoJIHZpZXdCb3g9IjAgMCAxNTQ2LjIgOTcwLjIiIHN0eWxlPSJlbmFibGUtYmFja2dyb3VuZDpuZXcgMCAwIDE1NDYuMiA5NzAuMjsiIHhtbDpzcGFjZT0icHJlc2VydmUiPgo8c3R5bGUgdHlwZT0idGV4dC9jc3MiPgoJLnN0MHtmaWxsOm5vbmU7c3Ryb2tlOiMzNzQxNTE7c3Ryb2tlLXdpZHRoOjcuODU0NTtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQoJLnN0MXtmaWxsOm5vbmU7c3Ryb2tlOiNFRjQ0NDQ7c3Ryb2tlLXdpZHRoOjEuOTYzNjtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQoJLnN0MntmaWxsOm5vbmU7c3Ryb2tlOiMzQjgyRjY7c3Ryb2tlLXdpZHRoOjEuOTYzNjtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQoJLnN0M3tmaWxsOm5vbmU7c3Ryb2tlOiNGQkJGMjQ7c3Ryb2tlLXdpZHRoOjEuOTYzNjtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQo8L3N0eWxlPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDAiIGQ9Ik01NTMuOCwyMjkuOGMzMy4xLDY2LjUsNjIuNiwxMzUuMiw5NC4zLDIwMi4zCgljMTAuNCwyMi4xLDIwLjMsNDUuNSwzMi40LDY2LjhjOC40LDE4LjYsMTguNCwzOSwzMy40LDUzYzEyLjcsMTUuMywzNi4yLDE5LjIsNTUsMTUuN2MxNy0zLjUsMzIuNC0xMi44LDQ3LjEtMjEuNgoJYzIyLjYtMTQuOCw0Ni40LTI4LjgsNjguNy00NC4yYzEwOS42LTc0LjMsMjguMy0xMzMsODMuNS0yMThjMjMuOS0zOC42LDY4LjktNjMsMTEzLjktNjUuOGMyMC44LTMuMSw0My0wLjcsNjMuOC0yLjkKCWMyNS4yLTEuOSw1MC40LTcuOCw3NC42LTE0LjdjNDAuNy0xMi45LDgwLjEtMzEuOCwxMTguOC00OS4xYzIxLjYtMTIuOCw1MC41LTE4LjIsNjEuOC00Mi4yYzExLjMtMjYuOC0wLjEtNTEuMSwzNy4zLTYwLjkKCWMxMy45LTMsMjcuNCwyLjYsNDAuMyw1LjljMTMuMSwzLjMsMjYuMyw0LjQsMzkuMywzLjljMTguMy00LjMsMjctMjUuMiwxMC44LTM5LjNjLTExLjUtOC42LTI2LjktOS45LTQwLjMtOS44CgljLTIzMiw3LjQtNDY1LjIsNC4zLTY5Ny4xLDcuOWMtMTQyLjYsMC43LTI4NSwxLjMtNDI3LjEsMWMtMTgsMC42LTMxLDE0LjctMjguNSwzMi40YzguMywxNi4xLDE2LjEsMzUuMiwzLjksNTEuMQoJYy01LjQsMTAuOC0xOC4zLDE2LjItMjcuNSwyMi42Yy0xNy42LDExLjctMzcuMywyMC42LTUzLDM0LjRjLTI1LjcsMTguNS00My43LDQ1LjItNjAuOSw3MC43Yy0yOC4yLDQyLjMtNDguOSw4OC43LTY2LjgsMTM1LjUKCUM3Ni44LDUwMy41LDMwLjQsNjQ3LjYsMTcuOCw3OTYuNGMtMy40LDM0LTguOSw2Ny4xLTguOCwxMDAuMWM1LjQsMzMuOSwyNS42LDMzLjksNTQsMzUuM2MyOC41LDguNywzMiwyOC45LDYzLjgsMjkuNAoJYzE5LjUtMC40LDM4LjctNy41LDU2LTEzLjhjMTkuMS02LjUsMzguNy0xOC4xLDQzLjItMzguM2M1LjUtMzIuNC0yMC4yLTY0LjctMzYuMy05MC4zYy0xNy4xLTI4LjUtMzQuMy01Ny4xLTUwLjEtODUuNAoJYy01MS4xLTg0LjQtMjYtMjA2LjIsNTktMjU3LjJjMjAuOC0xNi4xLDU1LjUtMzcuMiw3NC42LTkuOGM3LjYsMTIuNSw5LjMsMjcuMiwxMy43LDQwLjNjNS41LDIzLjksMjguMSw0MS44LDUyLjEsMzkuMgoJYzE5LjctMi4xLDM5LjMtOS42LDU2LTE4LjdjMzYuMy0xOC42LDMxLjEtNjUuMSwxNC43LTk1LjNjLTE4LjMtNDAuMi00Ni43LTc0LjktNjYuOC0xMTIuOWMtMTcuOC0yOC4yLTM3LjItNjUtMTkuNi05Ni4yCgljNS4yLTE1LjcsMTcuOC0yNy41LDI4LjUtMzguM2M5LjMtMTEsMTktMjIsMjguNS0zMS40YzI0LjUtMjguMiw1NS45LTcxLDk3LjItNTMuOWMyNy41LDEzLjksMzkuNCw0NS41LDUwLjEsNzEuNwoJQzUzNS4yLDE5MS41LDU0NS4xLDIxMC4yLDU1My44LDIyOS44eiIvPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDEiIGQ9Ik01NTMuOCwyMjkuOGMzNi44LDc0LjQsNjkuOSwxNTAuOCwxMDUuMSwyMjUuOAoJYzMzLjQsNjYuMiw1NywxNTAuMywxNDcuMyw5Ni4yYzUxLTM1LjEsMTIxLTYxLDEzOC40LTEyNi43YzYtMjcuNSwwLjMtNTUuOCwyLTgzLjVjNS45LTQ0LjcsMzQuMi04NC41LDczLjYtMTA2CgljNDMuMS0yNS40LDk1LTE1LjgsMTQyLjQtMjIuNmM1Ny4yLTcuNCwxMTEuMS0zMS42LDE2My01NiIvPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDIiIGQ9Ik0xMzI1LjUsMTU3LjJjMTMuOC01LjcsMjgtMTIuOSw0MS4yLTE5LjZjMTMuOC01LjcsMjguMS0xNC4zLDM0LjQtMjguNQoJYzctMTIuOSw0LTI5LjEsMTAuOC00Mi4yYzcuMS0xNCwyNi4xLTIxLjksNDEuMi0xOC43YzkuOS0wLjQsMTkuNyw1LjQsMjkuNSw2LjljMTMsMS45LDI3LjYsNS44LDQwLjMsMQoJYzI1LjgtMTEuNiwxNC41LTM5LjUtMTAuOC00NS4yYy0xNi40LTMuMy0zMy41LTEuNi01MC4xLTFjLTIxLjIsMC4yLTQyLjcsMS02My44LDJjLTMzOS45LDIuOC02ODAsNi4zLTEwMjAuMSw1LjkKCWMtMjYuNS0yLjktNTEsMTAuOC0zOS4zLDQwLjNjMjIuMiwzNC4yLTcuOSw1Ny0zNi4zLDcxLjdjLTM0LDE5LjQtNjYuOSw0My04OC40LDc2LjZjLTQyLjUsNTctNzAuMywxMjMtOTQuMywxODkuNQoJQzcwLjYsNTIxLjMsMzEuOCw2NTIsMTguNyw3ODYuNSIvPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDMiIGQ9Ik0xOC43LDc4Ni41Yy0yLjksMjMuNi00LjgsNDguMi03LjksNzEuN2MtMS41LDIyLTcuNiw0OS41LDExLjgsNjUuOAoJYzEwLDcuMSwyMi42LDcuMSwzNC40LDYuOWM3LjksMC44LDE1LjQsNSwyMi42LDcuOWMyNy42LDI1LjksNTAuNCwyNy40LDg1LjQsMTQuN2M0MC43LTEwLjUsNzYuOC0zMi44LDU2LTc5LjUKCWMtMTAuMy0yNi45LTI5LjItNDkuNy00My4yLTc0LjZjLTIwLjMtMzYuNC00Ni4zLTcxLjQtNTcuOS0xMTEuOWMtMTIuNS00NC40LTYuNS05My4xLDEwLjgtMTM1LjVjMjAuMi00MS45LDU3LjMtNzMuNSw5OC4yLTk0LjMKCWMyMi4yLTExLjgsNDcuMSwwLjQsNTEuMSwyNS41YzcuOCwzMCwxNiw2MS4yLDUzLDYyLjhjMTQuNiwwLjksMjguOS00LjgsNDIuMi05LjhjMjQtNi41LDQ2LjUtMjUuOSw0Ny4xLTUyCgljMS4yLTMzLjUtMTcuNS02My42LTM0LjQtOTEuM2MtMTcuNS0yOC4zLTM2LjEtNTYuMi01Mi04NS40Yy0xOS43LTMxLTI4LjUtNjYuNC01LjktOTguMmMxOC4yLTIzLDM5LjItNDQuMSw1OC45LTY1LjgKCWMxMi44LTE2LDI3LjYtMzEuNiw0NS4yLTQyLjJjMTkuNi05LjIsNDYuNC03LjcsNTkuOSwxMC44YzE2LjYsMTUuOCwyNSwzOC4xLDMzLjQsNTguOWM3LjYsMTkuOSwxNy44LDM5LjQsMjYuNSw1OC45Ii8+Cjwvc3ZnPgo='/></p>
        </h1>
        <h1><p> {row['name']}</p></h1>
        <p><p>Constructor information:</p>
        <ul>
            <li>Country name: {row['country']}</li>
        <details>
            <summary>More details</summary>
            {row['summary']}
        </details>     
        </ul>
        </p>
        """
        
        iframe = folium.IFrame(html=html, width=300, height=300)
        
        popup = folium.Popup(iframe, max_width=2650)
        
        folium.Marker(location=[row['lat'],row['long']],popup=popup,tooltip=row['summary'],radius=3, icon=folium.Icon(color='white')).add_to(my_map)

        
    return my_map

In [110]:
generate_map(circuits, drivers,constructors)

## Driver information

In [14]:
drivers

,DRIVER_ID,DRIVER,DRIVER_REF,DRIVER_CODE,DOB,NATIONALITY
0,1,Lewis Hamilton,hamilton,HAM,1985-01-07,British
1,3,Nico Rosberg,rosberg,ROS,1985-06-27,German
2,4,Fernando Alonso,alonso,ALO,1981-07-29,Spanish
3,8,Kimi Raikkonen,raikkonen,RAI,1979-10-17,Finnish
4,9,Robert Kubica,kubica,KUB,1984-12-07,Polish
5,13,Felipe Massa,massa,MAS,1981-04-25,Brazilian
6,16,Adrian Sutil,sutil,SUT,1983-01-11,German
7,18,Jenson Button,button,BUT,1980-01-19,British
8,20,Sebastian Vettel,vettel,VET,1987-07-03,German
9,154,Romain Grosjean,grosjean,GRO,1986-04-17,French


We need geographical data on the drivers, as well as a short contextualization of who they are to display on the map.

In [15]:
drivers['COUNTRY'] = drivers['NATIONALITY'].replace({'Andorran':'Andorra','Emirati':'United Arab Emirates','Afghan':'Afghanistan','Antiguan, Barbudan':'Antigua and Barbuda','Anguillian':'Anguilla','Albanian':'Albania','Armenian':'Armenia','Dutch':'Netherlands Antilles','Angolan':'Angola','???':'Antarctica','Argentinean':'Argentina','American Samoan':'American Samoa','Austrian':'Austria','Australian':'Australia','Aruban':'Aruba','Swedish':'Åland Islands','Azerbaijani':'Azerbaijan','Bosnian, Herzegovinian':'Bosnia and Herzegovina','Barbadian':'Barbados','Bangladeshi':'Bangladesh','Belgian':'Belgium','Burkinabe':'Burkina Faso','Bulgarian':'Bulgaria','Bahraini':'Bahrain','Burundian':'Burundi','Beninese':'Benin','Saint Barthélemy Islander':'Saint Barthélemy','Bermudian':'Bermuda','Bruneian':'Brunei','Bolivian':'Bolivia','Dutch':'British Antarctic Territory','Brazilian':'Brazil','Bahamian':'Bahamas','Bhutanese':'Bhutan','???':'Bouvet Island','Motswana':'Botswana','Belarusian':'Belarus','Belizean':'Belize','Canadian':'Canada','Cocos Islander':'Cocos [Keeling] Islands','Congolese':'Congo - Kinshasa','Central African':'Central African Republic','Congolese':'Congo - Brazzaville','Swiss':'Switzerland','Ivorian':'Côte d’Ivoire','Cook Islander':'Cook Islands','Chilean':'Chile','Cameroonian':'Cameroon','Chinese':'China','Colombian':'Colombia','Costa Rican':'Costa Rica','Montenegrins, Serbs':'Serbia and Montenegro','???':'Canton and Enderbury Islands','Cuban':'Cuba','Cape Verdian':'Cape Verde','Curaçaoan':'Curaçao','Christmas Island':'Christmas Island','Cypriot':'Cyprus','Czech':'Czech Republic','German':'Germany','Djibouti':'Djibouti','Danish':'Denmark','Dominican':'Dominica','Dominican':'Dominican Republic','Algerian':'Algeria','Ecuadorean':'Ecuador','Estonian':'Estonia','Egyptian':'Egypt','Sahrawi':'Western Sahara','Eritrean':'Eritrea','Spanish':'Spain','Ethiopian':'Ethiopia','Finnish':'Finland','Fijian':'Fiji','Falkland Islander':'Falkland Islands','Micronesian':'Micronesia','Faroese':'Faroe Islands','???':'French Southern and Antarctic Territories','French':'France','???':'Metropolitan France','Gabonese':'Gabon','British':'United Kingdom','Grenadian':'Grenada','Georgian':'Georgia','???':'French Guiana','Channel Islander':'Guernsey','Ghanaian':'Ghana','Gibraltar':'Gibraltar','Greenlandic':'Greenland','Gambian':'Gambia','Guinean':'Guinea','Guadeloupian':'Guadeloupe','Equatorial Guinean':'Equatorial Guinea','Greek':'Greece','South Georgia and the South Sandwich Islander':'South Georgia and the South Sandwich Islands','Guatemalan':'Guatemala','Guamanian':'Guam','Guinea-Bissauan':'Guinea-Bissau','Guyanese':'Guyana','Chinese':'Hong Kong SAR China','Heard and McDonald Islander':'Heard Island and McDonald Islands','Honduran':'Honduras','Croatian':'Croatia','Haitian':'Haiti','Hungarian':'Hungary','Indonesian':'Indonesia','Irish':'Ireland','Israeli':'Israel','Manx':'Isle of Man','Indian':'India','Indian':'British Indian Ocean Territory','Iraqi':'Iraq','Iranian':'Iran','Icelander':'Iceland','Italian':'Italy','Channel Islander':'Jersey','Jamaican':'Jamaica','Jordanian':'Jordan','Japanese':'Japan','???':'Johnston Island','Kenyan':'Kenya','Kirghiz':'Kyrgyzstan','Cambodian':'Cambodia','I-Kiribati':'Kiribati','Comoran':'Comoros','Kittian and Nevisian':'Saint Kitts and Nevis','North Korean':'North Korea','South Korean':'South Korea','Kuwaiti':'Kuwait','Caymanian':'Cayman Islands','Kazakhstani':'Kazakhstan','Laotian':'Laos','Lebanese':'Lebanon','Saint Lucian':'Saint Lucia','Liechtensteiner':'Liechtenstein','Sri Lankan':'Sri Lanka','Liberian':'Liberia','Mosotho':'Lesotho','Lithuanian':'Lithuania','Luxembourger':'Luxembourg','Latvian':'Latvia','Libyan':'Libya','Moroccan':'Morocco','Monegasque':'Monaco','Moldovan':'Moldova','Montenegrin':'Montenegro','Saint Martin Islander':'Saint Martin','Malagasy':'Madagascar','Marshallese':'Marshall Islands','???':'Midway Islands','Macedonian':'Macedonia','Malian':'Mali','Myanmar':'Myanmar [Burma]','Mongolian':'Mongolia','Chinese':'China','American':'Northern Mariana Islands','French':'Martinique','Mauritanian':'Mauritania','Montserratian':'Montserrat','Maltese':'Malta','Mauritian':'Mauritius','Maldivan':'Maldives','Malawian':'Malawi','Mexican':'Mexico','Malaysian':'Malaysia','Mozambican':'Mozambique','Namibian':'Namibia','New Caledonian':'New Caledonia','Nigerian':'Niger','Norfolk Islander':'Norfolk Island','Nigerian':'Nigeria','Nicaraguan':'Nicaragua','Dutch':'Netherlands','Norwegian':'Norway','Nepalese':'Nepal','???':'Dronning Maud Land','Nauruan':'Nauru','???':'Neutral Zone','Niuean':'Niue','New Zealander':'New Zealand','Omani':'Oman','Panamanian':'Panama','???':'Pacific Islands Trust Territory','Peruvian':'Peru','French Polynesian':'French Polynesia','Papua New Guinean':'Papua New Guinea','Filipino':'Philippines','Pakistani':'Pakistan','Polish':'Poland','French':'Saint Pierre and Miquelon','Pitcairn Islander':'Pitcairn Islands','Puerto Rican':'Puerto Rico','Palestinian':'Palestinian Territories','Portuguese':'Portugal','???':'U.S. Miscellaneous Pacific Islands','Palauan':'Palau','Paraguayan':'Paraguay','???':'Panama Canal Zone','Qatari':'Qatar','French':'Réunion','Romanian':'Romania','Serbian':'Serbia','Russian':'Russia','Rwandan':'Rwanda','Saudi Arabian':'Saudi Arabia','Solomon Islander':'Solomon Islands','Seychellois':'Seychelles','Sudanese':'Sudan','Swedish':'Sweden','Singaporean':'Singapore','Saint Helenian':'Saint Helena','Slovene':'Slovenia','Norwegian':'Svalbard and Jan Mayen','Slovak':'Slovakia','Sierra Leonean':'Sierra Leone','Sammarinese':'San Marino','Senegalese':'Senegal','Somali':'Somalia','Surinamer':'Suriname','Sao Tomean':'São Tomé and Príncipe','???':'Union of Soviet Socialist Republics','Salvadoran':'El Salvador','Syrian':'Syria','Swazi':'Swaziland','Turks and Caicos Islander':'Turks and Caicos Islands','Chadian':'Chad','French':'French Southern Territories','Togolese':'Togo','Thai':'Thailand','Tadzhik':'Tajikistan','Tokelauan':'Tokelau','East Timorese':'Timor-Leste','Turkmen':'Turkmenistan','Tunisian':'Tunisia','Tongan':'Tonga','Turkish':'Turkey','Trinidadian':'Trinidad and Tobago','Tuvaluan':'Tuvalu','Taiwanese':'Taiwan','Tanzanian':'Tanzania','Ukrainian':'Ukraine','Ugandan':'Uganda','American':'U.S. Minor Outlying Islands','American':'United States','Uruguayan':'Uruguay','Uzbekistani':'Uzbekistan','Italian':'Vatican City','Saint Vincentian':'Saint Vincent and the Grenadines','???':'North Vietnam','Venezuelan':'Venezuela','Virgin Islander':'British Virgin Islands','Virgin Islander':'U.S. Virgin Islands','Vietnamese':'Vietnam','Ni-Vanuatu':'Vanuatu','Wallis and Futuna Islander':'Wallis and Futuna','???':'Wake Island','Samoan':'Samoa','Yemeni':'Yemen','French':'France','South African':'South Africa','Zambian':'Zambia','Zimbabwean':'Zimbabwe'}
)

In [17]:
drivers.replace({"Carlos Sainz":"Carlos Sainz Jr."}, inplace = True)

In [40]:
drivers['SUMMARY'] = get_wiki_summary(drivers['DRIVER'])

C:\Users\ricar\anaconda3\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\ricar\anaconda3\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


George Russell


There was an error inputting the information regarding George Russel. This is due to the fact that there are multiple George Russels on Wikipedia. To solve this quickly, we will replace the text with wikipedia's summary.

In [19]:
drivers.replace({'error':'George William Russell (/rʌsəl/; born 15 February 1998) is a British racing driver currently competing in Formula One for Mercedes. He was the 2018 FIA Formula 2 Champion for ART and the 2017 GP3 Series Champion. Following his Formula 2 championship win, Russell signed for Williams in 2019, making his debut at the 2019 Australian Grand Prix, although he stood in for Lewis Hamilton at Mercedes at the 2020 Sakhir Grand Prix. Russell achieved his first F1 podium with Williams at the 2021 Belgian Grand Prix. He is also part of the Mercedes Young Driver programme.'}, inplace = True)

In [47]:
drivers['lat'],drivers['long'] = get_coords(drivers['COUNTRY'])

In [48]:
drivers

,DRIVER_ID,DRIVER,DRIVER_REF,DRIVER_CODE,DOB,NATIONALITY,COUNTRY,SUMMARY,lat,long
0,1,Lewis Hamilton,hamilton,HAM,1985-01-07,British,United Kingdom,Sir Lewis Carl Davidson Hamilton (born 7 Janu...,55,-3
1,55.73943271643605,Nico Rosberg,rosberg,ROS,1985-06-27,German,Germany,Nico Erik Rosberg (born 27 June 1985) is a Ger...,51,9
2,37.659451800424335,Fernando Alonso,alonso,ALO,1981-07-29,Spanish,Spain,Fernando Alonso Díaz (Spanish pronunciation: [...,40,-4
3,8,Kimi Raikkonen,raikkonen,RAI,1979-10-17,Finnish,Finland,Kimi-Matias Räikkönen (Finnish pronunciation: ...,64,26
4,9,Robert Kubica,kubica,KUB,1984-12-07,Polish,Poland,Robert Józef Kubica (Polish pronunciation: [ˈr...,52,20
5,13,Felipe Massa,massa,MAS,1981-04-25,Brazilian,Brazil,Felipe Massa (Portuguese pronunciation: [fiˈli...,-10,-52
6,16,Adrian Sutil,sutil,SUT,1983-01-11,German,Germany,Adrian Sutil (German pronunciation: [ˈadɾjan z...,51,9
7,18,Jenson Button,button,BUT,1980-01-19,British,United Kingdom,Jenson Alexander Lyons Button (born 19 Januar...,55,-3
8,20,Sebastian Vettel,vettel,VET,1987-07-03,German,Germany,Sebastian Vettel (German pronunciation: [zeˈba...,51,9
9,154,Romain Grosjean,grosjean,GRO,1986-04-17,French,France,Romain Grosjean (French: [ʁɔmɛ̃ ɡʁoʒɑ̃]; born ...,47,2


Manually input the right lat & long values for Russia

In [49]:
drivers

,DRIVER_ID,DRIVER,DRIVER_REF,DRIVER_CODE,DOB,NATIONALITY,COUNTRY,SUMMARY,lat,long
0,1,Lewis Hamilton,hamilton,HAM,1985-01-07,British,United Kingdom,Sir Lewis Carl Davidson Hamilton (born 7 Janu...,55,-3
1,55.73943271643605,Nico Rosberg,rosberg,ROS,1985-06-27,German,Germany,Nico Erik Rosberg (born 27 June 1985) is a Ger...,51,9
2,37.659451800424335,Fernando Alonso,alonso,ALO,1981-07-29,Spanish,Spain,Fernando Alonso Díaz (Spanish pronunciation: [...,40,-4
3,8,Kimi Raikkonen,raikkonen,RAI,1979-10-17,Finnish,Finland,Kimi-Matias Räikkönen (Finnish pronunciation: ...,64,26
4,9,Robert Kubica,kubica,KUB,1984-12-07,Polish,Poland,Robert Józef Kubica (Polish pronunciation: [ˈr...,52,20
5,13,Felipe Massa,massa,MAS,1981-04-25,Brazilian,Brazil,Felipe Massa (Portuguese pronunciation: [fiˈli...,-10,-52
6,16,Adrian Sutil,sutil,SUT,1983-01-11,German,Germany,Adrian Sutil (German pronunciation: [ˈadɾjan z...,51,9
7,18,Jenson Button,button,BUT,1980-01-19,British,United Kingdom,Jenson Alexander Lyons Button (born 19 Januar...,55,-3
8,20,Sebastian Vettel,vettel,VET,1987-07-03,German,Germany,Sebastian Vettel (German pronunciation: [zeˈba...,51,9
9,154,Romain Grosjean,grosjean,GRO,1986-04-17,French,France,Romain Grosjean (French: [ʁɔmɛ̃ ɡʁoʒɑ̃]; born ...,47,2


## Constructor information

In [78]:
constructors

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
2,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
3,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso
4,6,ferrari,Ferrari,Italian,http://en.wikipedia.org/wiki/Scuderia_Ferrari
5,9,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing
6,10,force_india,Force India,Indian,http://en.wikipedia.org/wiki/Racing_Point_Forc...
7,15,sauber,Sauber,Swiss,http://en.wikipedia.org/wiki/Sauber
8,51,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...
9,117,aston_martin,Aston Martin,British,http://en.wikipedia.org/wiki/Aston_Martin_in_F...


In [87]:
constructors['country'] = constructors['nationality'].replace({'Andorran':'Andorra','Emirati':'United Arab Emirates','Afghan':'Afghanistan','Antiguan, Barbudan':'Antigua and Barbuda','Anguillian':'Anguilla','Albanian':'Albania','Armenian':'Armenia','Dutch':'Netherlands Antilles','Angolan':'Angola','???':'Antarctica','Argentinean':'Argentina','American Samoan':'American Samoa','Austrian':'Austria','Australian':'Australia','Aruban':'Aruba','Swedish':'Åland Islands','Azerbaijani':'Azerbaijan','Bosnian, Herzegovinian':'Bosnia and Herzegovina','Barbadian':'Barbados','Bangladeshi':'Bangladesh','Belgian':'Belgium','Burkinabe':'Burkina Faso','Bulgarian':'Bulgaria','Bahraini':'Bahrain','Burundian':'Burundi','Beninese':'Benin','Saint Barthélemy Islander':'Saint Barthélemy','Bermudian':'Bermuda','Bruneian':'Brunei','Bolivian':'Bolivia','Dutch':'British Antarctic Territory','Brazilian':'Brazil','Bahamian':'Bahamas','Bhutanese':'Bhutan','???':'Bouvet Island','Motswana':'Botswana','Belarusian':'Belarus','Belizean':'Belize','Canadian':'Canada','Cocos Islander':'Cocos [Keeling] Islands','Congolese':'Congo - Kinshasa','Central African':'Central African Republic','Congolese':'Congo - Brazzaville','Swiss':'Switzerland','Ivorian':'Côte d’Ivoire','Cook Islander':'Cook Islands','Chilean':'Chile','Cameroonian':'Cameroon','Chinese':'China','Colombian':'Colombia','Costa Rican':'Costa Rica','Montenegrins, Serbs':'Serbia and Montenegro','???':'Canton and Enderbury Islands','Cuban':'Cuba','Cape Verdian':'Cape Verde','Curaçaoan':'Curaçao','Christmas Island':'Christmas Island','Cypriot':'Cyprus','Czech':'Czech Republic','German':'Germany','Djibouti':'Djibouti','Danish':'Denmark','Dominican':'Dominica','Dominican':'Dominican Republic','Algerian':'Algeria','Ecuadorean':'Ecuador','Estonian':'Estonia','Egyptian':'Egypt','Sahrawi':'Western Sahara','Eritrean':'Eritrea','Spanish':'Spain','Ethiopian':'Ethiopia','Finnish':'Finland','Fijian':'Fiji','Falkland Islander':'Falkland Islands','Micronesian':'Micronesia','Faroese':'Faroe Islands','???':'French Southern and Antarctic Territories','French':'France','???':'Metropolitan France','Gabonese':'Gabon','British':'United Kingdom','Grenadian':'Grenada','Georgian':'Georgia','???':'French Guiana','Channel Islander':'Guernsey','Ghanaian':'Ghana','Gibraltar':'Gibraltar','Greenlandic':'Greenland','Gambian':'Gambia','Guinean':'Guinea','Guadeloupian':'Guadeloupe','Equatorial Guinean':'Equatorial Guinea','Greek':'Greece','South Georgia and the South Sandwich Islander':'South Georgia and the South Sandwich Islands','Guatemalan':'Guatemala','Guamanian':'Guam','Guinea-Bissauan':'Guinea-Bissau','Guyanese':'Guyana','Chinese':'Hong Kong SAR China','Heard and McDonald Islander':'Heard Island and McDonald Islands','Honduran':'Honduras','Croatian':'Croatia','Haitian':'Haiti','Hungarian':'Hungary','Indonesian':'Indonesia','Irish':'Ireland','Israeli':'Israel','Manx':'Isle of Man','Indian':'India','Indian':'India','Iraqi':'Iraq','Iranian':'Iran','Icelander':'Iceland','Italian':'Italy','Channel Islander':'Jersey','Jamaican':'Jamaica','Jordanian':'Jordan','Japanese':'Japan','???':'Johnston Island','Kenyan':'Kenya','Kirghiz':'Kyrgyzstan','Cambodian':'Cambodia','I-Kiribati':'Kiribati','Comoran':'Comoros','Kittian and Nevisian':'Saint Kitts and Nevis','North Korean':'North Korea','South Korean':'South Korea','Kuwaiti':'Kuwait','Caymanian':'Cayman Islands','Kazakhstani':'Kazakhstan','Laotian':'Laos','Lebanese':'Lebanon','Saint Lucian':'Saint Lucia','Liechtensteiner':'Liechtenstein','Sri Lankan':'Sri Lanka','Liberian':'Liberia','Mosotho':'Lesotho','Lithuanian':'Lithuania','Luxembourger':'Luxembourg','Latvian':'Latvia','Libyan':'Libya','Moroccan':'Morocco','Monegasque':'Monaco','Moldovan':'Moldova','Montenegrin':'Montenegro','Saint Martin Islander':'Saint Martin','Malagasy':'Madagascar','Marshallese':'Marshall Islands','???':'Midway Islands','Macedonian':'Macedonia','Malian':'Mali','Myanmar':'Myanmar [Burma]','Mongolian':'Mongolia','Chinese':'China','American':'Northern Mariana Islands','French':'Martinique','Mauritanian':'Mauritania','Montserratian':'Montserrat','Maltese':'Malta','Mauritian':'Mauritius','Maldivan':'Maldives','Malawian':'Malawi','Mexican':'Mexico','Malaysian':'Malaysia','Mozambican':'Mozambique','Namibian':'Namibia','New Caledonian':'New Caledonia','Nigerian':'Niger','Norfolk Islander':'Norfolk Island','Nigerian':'Nigeria','Nicaraguan':'Nicaragua','Dutch':'Netherlands','Norwegian':'Norway','Nepalese':'Nepal','???':'Dronning Maud Land','Nauruan':'Nauru','???':'Neutral Zone','Niuean':'Niue','New Zealander':'New Zealand','Omani':'Oman','Panamanian':'Panama','???':'Pacific Islands Trust Territory','Peruvian':'Peru','French Polynesian':'French Polynesia','Papua New Guinean':'Papua New Guinea','Filipino':'Philippines','Pakistani':'Pakistan','Polish':'Poland','French':'Saint Pierre and Miquelon','Pitcairn Islander':'Pitcairn Islands','Puerto Rican':'Puerto Rico','Palestinian':'Palestinian Territories','Portuguese':'Portugal','???':'U.S. Miscellaneous Pacific Islands','Palauan':'Palau','Paraguayan':'Paraguay','???':'Panama Canal Zone','Qatari':'Qatar','French':'Réunion','Romanian':'Romania','Serbian':'Serbia','Russian':'Russia','Rwandan':'Rwanda','Saudi Arabian':'Saudi Arabia','Solomon Islander':'Solomon Islands','Seychellois':'Seychelles','Sudanese':'Sudan','Swedish':'Sweden','Singaporean':'Singapore','Saint Helenian':'Saint Helena','Slovene':'Slovenia','Norwegian':'Svalbard and Jan Mayen','Slovak':'Slovakia','Sierra Leonean':'Sierra Leone','Sammarinese':'San Marino','Senegalese':'Senegal','Somali':'Somalia','Surinamer':'Suriname','Sao Tomean':'São Tomé and Príncipe','???':'Union of Soviet Socialist Republics','Salvadoran':'El Salvador','Syrian':'Syria','Swazi':'Swaziland','Turks and Caicos Islander':'Turks and Caicos Islands','Chadian':'Chad','French':'French Southern Territories','Togolese':'Togo','Thai':'Thailand','Tadzhik':'Tajikistan','Tokelauan':'Tokelau','East Timorese':'Timor-Leste','Turkmen':'Turkmenistan','Tunisian':'Tunisia','Tongan':'Tonga','Turkish':'Turkey','Trinidadian':'Trinidad and Tobago','Tuvaluan':'Tuvalu','Taiwanese':'Taiwan','Tanzanian':'Tanzania','Ukrainian':'Ukraine','Ugandan':'Uganda','American':'U.S. Minor Outlying Islands','American':'United States','Uruguayan':'Uruguay','Uzbekistani':'Uzbekistan','Italy':'Italian','Saint Vincentian':'Saint Vincent and the Grenadines','???':'North Vietnam','Venezuelan':'Venezuela','Virgin Islander':'British Virgin Islands','Virgin Islander':'U.S. Virgin Islands','Vietnamese':'Vietnam','Ni-Vanuatu':'Vanuatu','Wallis and Futuna Islander':'Wallis and Futuna','???':'Wake Island','Samoan':'Samoa','Yemeni':'Yemen','French':'France','South African':'South Africa','Zambian':'Zambia','Zimbabwean':'Zimbabwe'}
)
constructors

,constructorId,constructorRef,name,nationality,url,country,lat,long
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,United Kingdom,55,-3
1,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,United Kingdom,55,-3
2,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...,France,47,2
3,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso,Italy,3,4
4,6,ferrari,Ferrari,Italian,http://en.wikipedia.org/wiki/Scuderia_Ferrari,Italy,3,4
5,9,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing,Austria,47.3333333300000020926745492033660411834716796875,13.3333333300000003163177098031155765056610107...
6,10,force_india,Force India,Indian,http://en.wikipedia.org/wiki/Racing_Point_Forc...,India,21,78
7,15,sauber,Sauber,Swiss,http://en.wikipedia.org/wiki/Sauber,Switzerland,46.8333333300000020926745492033660411834716796875,8.33333333000000031631770980311557650566101074...
8,51,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Switzerland,46.8333333300000020926745492033660411834716796875,8.33333333000000031631770980311557650566101074...
9,117,aston_martin,Aston Martin,British,http://en.wikipedia.org/wiki/Aston_Martin_in_F...,United Kingdom,55,-3


In [98]:
constructors['lat'],constructors['long'] = get_coords(constructors['country'])

In [99]:
constructors['summary'] = get_wiki_summary(constructors['name'])

Williams
Mercedes
AlphaTauri


We need to manually input the information for Williams, Mercedes and AlphaTauri

In [ ]:
#constructors.replace()

In [100]:
constructors

,constructorId,constructorRef,name,nationality,url,country,lat,long,summary
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,United Kingdom,55,-3,McLaren Racing Limited is a British motor raci...
1,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,United Kingdom,55,-3,error
2,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...,France,47,2,"Groupe Renault (UK: REN-oh, US: rə-NAWLT, rə..."
3,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso,Italy,43,12,"Scuderia Toro Rosso, commonly known as Toro Ro..."
4,6,ferrari,Ferrari,Italian,http://en.wikipedia.org/wiki/Scuderia_Ferrari,Italy,43,12,Ferrari S.p.A. (; Italian: [ferˈraːri]) is an ...
5,9,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing,Austria,47.3333333300000020926745492033660411834716796875,13.3333333300000003163177098031155765056610107...,Red Bull is a brand of energy drinks sold by A...
6,10,force_india,Force India,Indian,http://en.wikipedia.org/wiki/Racing_Point_Forc...,India,21,78,"Force India Formula One Team Limited, commonly..."
7,15,sauber,Sauber,Swiss,http://en.wikipedia.org/wiki/Sauber,Switzerland,46.8333333300000020926745492033660411834716796875,8.33333333000000031631770980311557650566101074...,Sauber Motorsport AG is a Swiss motorsport eng...
8,51,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Switzerland,46.8333333300000020926745492033660411834716796875,8.33333333000000031631770980311557650566101074...,Alfa Romeo Automobiles S.p.A. (Italian: [ˈalfa...
9,117,aston_martin,Aston Martin,British,http://en.wikipedia.org/wiki/Aston_Martin_in_F...,United Kingdom,55,-3,Aston Martin Lagonda is a British independent ...


## Circuit information

In [50]:
circuits

,circuitId,name,location,country,alt,url,summary
0,1,Albert Park Grand Prix Circuit,Melbourne,Australia,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,The Albert Park Circuit is a motorsport street...
1,2,Sepang International Circuit,Kuala Lumpur,Malaysia,18.0,http://en.wikipedia.org/wiki/Sepang_Internatio...,The Sepang International Circuit (Malay: Litar...
2,3,Bahrain International Circuit,Sakhir,Bahrain,7.0,http://en.wikipedia.org/wiki/Bahrain_Internati...,The Bahrain International Circuit (Arabic: حلب...
3,4,Circuit de Barcelona-Catalunya,Montmeló,Spain,109.0,http://en.wikipedia.org/wiki/Circuit_de_Barcel...,The Circuit de Barcelona-Catalunya (Catalan pr...
4,5,Istanbul Park,Istanbul,Turkey,130.0,http://en.wikipedia.org/wiki/Istanbul_Park,Intercity Istanbul Park (Turkish: İstanbul Par...
5,6,Circuit de Monaco,Monte-Carlo,Monaco,7.0,http://en.wikipedia.org/wiki/Circuit_de_Monaco,Circuit de Monaco is a 3.337 km (2.074 mi) str...
6,7,Circuit Gilles Villeneuve,Montreal,Canada,13.0,http://en.wikipedia.org/wiki/Circuit_Gilles_Vi...,The Circuit Gilles Villeneuve (also spelled Ci...
7,9,Silverstone Circuit,Silverstone,UK,153.0,http://en.wikipedia.org/wiki/Silverstone_Circuit,Silverstone Circuit is a motor racing circuit ...
8,10,Hockenheimring,Hockenheim,Germany,103.0,http://en.wikipedia.org/wiki/Hockenheimring,The Hockenheimring Baden-Württemberg (German: ...
9,11,Hungaroring,Budapest,Hungary,264.0,http://en.wikipedia.org/wiki/Hungaroring,The Hungaroring is a 4.381 km (2.722 mi) motor...


There is an issue with our coordinates. Thus, we will leverage on Wikipedia module's functionality to get new coordinates.

In [83]:
circuits.drop(columns=['lat','lng'],inplace=True)

KeyError: "['lng'] not found in axis"

In [52]:
circuits['lat'], circuits['long'] = get_coords(circuits['name'])

We can extend the information we display to the map's users by adding a spoiler feature within the popup of the map, containing Wikipedia's summary for the entry.

In [53]:
circuits['summary'] = get_wiki_summary(circuits['name'])

In [54]:
circuits

,circuitId,name,location,country,alt,url,summary,lat,long
0,1,Albert Park Grand Prix Circuit,Melbourne,Australia,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,The Albert Park Circuit is a motorsport street...,-37.849722219999996752903825836256146430969238...,144.96833333000000720858224667608737945556640625
1,2,Sepang International Circuit,Kuala Lumpur,Malaysia,18.0,http://en.wikipedia.org/wiki/Sepang_Internatio...,The Sepang International Circuit (Malay: Litar...,2.76055555999999979732706378854345530271530151...,101.737499999999997157829056959599256515502929...
2,3,Bahrain International Circuit,Sakhir,Bahrain,7.0,http://en.wikipedia.org/wiki/Bahrain_Internati...,The Bahrain International Circuit (Arabic: حلب...,26.032499999999998863131622783839702606201171875,50.51055556000000024141627363860607147216796875
3,4,Circuit de Barcelona-Catalunya,Montmeló,Spain,109.0,http://en.wikipedia.org/wiki/Circuit_de_Barcel...,The Circuit de Barcelona-Catalunya (Catalan pr...,41.5700000000000002842170943040400743484497070...,2.26111110999999986859165801433846354484558105...
4,5,Istanbul Park,Istanbul,Turkey,130.0,http://en.wikipedia.org/wiki/Istanbul_Park,Intercity Istanbul Park (Turkish: İstanbul Par...,40.9516666700000016021476767491549253463745117...,29.405000000000001136868377216160297393798828125
5,6,Circuit de Monaco,Monte-Carlo,Monaco,7.0,http://en.wikipedia.org/wiki/Circuit_de_Monaco,Circuit de Monaco is a 3.337 km (2.074 mi) str...,43.7347222200000018688115233089774847030639648...,7.42055556000000038352482079062610864639282226...
6,7,Circuit Gilles Villeneuve,Montreal,Canada,13.0,http://en.wikipedia.org/wiki/Circuit_Gilles_Vi...,The Circuit Gilles Villeneuve (also spelled Ci...,45.5005777800000004162939148955047130584716796875,-73.522461109999994732788763940334320068359375
7,9,Silverstone Circuit,Silverstone,UK,153.0,http://en.wikipedia.org/wiki/Silverstone_Circuit,Silverstone Circuit is a motor racing circuit ...,52.0786111099999970974749885499477386474609375,-1.0169444400000000605643890594365075230598449...
8,10,Hockenheimring,Hockenheim,Germany,103.0,http://en.wikipedia.org/wiki/Hockenheimring,The Hockenheimring Baden-Württemberg (German: ...,49.3277777799999981311884766910225152969360351...,8.56583333000000024526343622710555791854858398...
9,11,Hungaroring,Budapest,Hungary,264.0,http://en.wikipedia.org/wiki/Hungaroring,The Hungaroring is a 4.381 km (2.722 mi) motor...,47.5822222199999984582063916604965925216674804...,19.2511111100000000817544787423685193061828613...


## Generating the map 

In [ ]:
generate_map(circuits)

In [ ]:
circuits